In [2]:
# importing libraries
import cv2
import numpy as np
import os

In [3]:
### KNN CODE
def distance(v1,v2):
    # Eucledian
    return np.sqrt(((v1-v2)**2).sum())
def knn(train,test,k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        # taking vector and label
        ix = train[i,:-1]
        iy = train[i,-1]
        # dist from test point
        d = distance(test,ix)
        dist.append([d,iy])
    # sorting based on distance and get top k elements
    dk = sorted(dist,key=lambda x:x[0])[:k]
    # retrieve labels
    labels = np.array(dk)[:,-1]
    # freq of each label
    output = np.unique(labels,return_counts=True)
    # MAX FREQ IS SELECTED
    index = np.argmax(output[1])
    return output[0][index]
##################################

In [4]:
# pipeline establishment and importing haarcascade
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("C:/Users/TARUN KUMAR/Downloads/haarcascade_frontalface_alt.xml")
skip = 0

face_data = []
labels = []

dataset_path = 'C:/Users/TARUN KUMAR/OPEN_CV/'
class_id = 0
names = {}

In [5]:
# Data_prepration
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        # create labels for class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

Loaded DADy.npy
Loaded deepanshi.npy
Loaded hajoor ji.npy
Loaded hira lal.npy
Loaded jay shetty.npy
Loaded rishma.npy
Loaded tarun.npy
(204, 30000)
(204, 1)
(204, 30001)


In [7]:
# testing
type(trainset)
trainset


array([[ 79.,  90.,  46., ..., 234., 189.,   0.],
       [ 30.,  36.,   0., ..., 206., 163.,   0.],
       [  2.,   4.,   0., ..., 193., 171.,   0.],
       ...,
       [  1.,   4.,   0., ...,   1.,   1.,   6.],
       [  3.,  24.,   8., ...,   2.,   2.,   6.],
       [ 64., 102.,  85., ...,   1.,   2.,   6.]])

In [ ]:
while True:
    ret,frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    if len(faces)==0:
        continue
    
    for face in faces:
        x,y,w,h = face
        # GET THE face ROI
        offset = 10
        face_section  = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        if np.all(np.array(face_section.shape)):
            face_section = cv2.resize(face_section,(100,100))
            # Predicted Label (out)
            out = knn(trainset,face_section.flatten())
            # Display on screen
            pred_name = names[int(out)]
            cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    cv2.imshow("FACE CAM",frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()